In [1]:
# Setting to adjust before each run:
MODEL_NAME = 'V3'
CODE_ENV = 'aws' #'kaggle', 'aws', 'local'
TEST_END  = 1941 #1969 

In [2]:
#Import data handling libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Masking, RepeatVector, Dropout, Reshape
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras import backend as K
from keras.callbacks import Callback
import tensorflow as tf

/tmp/ipykernel_13248/3405039513.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-02 20:35:43.826586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-02 20:35:44.782138: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())

Num GPUs Available:  1
True


In [4]:
#Specify directories
if CODE_ENV=='local':
    ###local###
    #get parent folder of current directory
    parent_dir = '/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction'

    #Directory resources
    res_dir = parent_dir + '/res/'
    src_dir = parent_dir + '/src/'
    prc_dir = src_dir + 'processed_data/' # Processed data directory with pickled dataframes

if CODE_ENV=='kaggle':
    ###On Kaggle###
    res_dir = '/kaggle/input/m5-forecasting-accuracy/'
    prc_dir = '/kaggle/input/processed-data-v3/'

if CODE_ENV=='aws':
    parent_dir = '/home/ubuntu/projects/Kaggle_M5PointPrediction'
    res_dir = parent_dir + '/res/'
    src_dir = parent_dir + '/src/'
    prc_dir = src_dir + 'processed_data/' # Processed data directory with pickled dataframes

In [5]:
# Create variables
BASE      = prc_dir +'df_1.pkl'
CALENDAR  = prc_dir +'df_2.pkl'
NUM_ITEMS = 30490 # Number of items per each day
# Set time_steps for defining test, train and validation sets
DAYS_PER_SEQUENCE = 7  # Length of the sequence
target_col = 'sales_amount'

In [6]:
# Read in df_train_conv from pickle file
df_all_data = pd.concat([pd.read_pickle(BASE),
           pd.read_pickle(CALENDAR)], 
           axis=1)

In [7]:
# Define categorical and numerical columns
categorical_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'is_available',
                    'd', 'wday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
                    'snap_CA', 'snap_TX', 'snap_WI', 'mday', 'week', 'month', 'year']
numerical_cols = ['sell_price']

# Convert categorical columns to category dtype and encode with cat.codes
for col in categorical_cols:
    df_all_data[col] = df_all_data[col].astype('category').cat.codes

# Normalize numerical columns
scaler = MinMaxScaler()
df_all_data[numerical_cols] = scaler.fit_transform(df_all_data[numerical_cols].astype(np.float32))
df_all_data[target_col] = scaler.fit_transform(df_all_data[[target_col]].astype(np.float32))

In [8]:
# Splitting the data in train, validation and test set; days are now 0 based, so have to shift by 1
# Define duration in days of each set
VAL_DUR   = 28
TEST_DUR  = 28

# Define end days of training set for each set
VAL_END   = TEST_END - TEST_DUR
TRAIN_END = VAL_END - VAL_DUR # 1885 -> Train only until the 28 days before the end of the data

# Finally define duration in days for the train set
TRAIN_DUR = TRAIN_END - DAYS_PER_SEQUENCE# Depends on whether the whole dataset is used or last the 28 days for validation 

df_train = df_all_data[df_all_data['d'] < TRAIN_END].reset_index(drop=True)
df_val   = df_all_data[(df_all_data['d'] >= TRAIN_END - DAYS_PER_SEQUENCE) & (df_all_data['d'] < VAL_END)].reset_index(drop=True) #35 days because of the time_steps shift
df_test  = df_all_data[(df_all_data['d'] >= VAL_END - DAYS_PER_SEQUENCE)   & (df_all_data['d'] < TEST_END)].reset_index(drop=True) #35 days because of the time_steps shift

# Delete df_all_data to free up memory as data is now stored in df_train, df_val and df_test
del df_all_data

In [9]:
# def lstm_data_generator(df, num_features, target, days_sliding_window, batch_size, once_only_features, repeated_features):
#     while True:
#         length_days = len(df) // NUM_ITEMS #1941 days
#         for i in range(0, length_days - days_sliding_window):
#             batch_sequences = []
#             batch_targets = []

#             start_ind = i * NUM_ITEMS
#             end_ind = start_ind + NUM_ITEMS * (days_sliding_window + 1) # predict the next day after the sequence

#             # Initialize an array to hold the features for each day
#             day_sequences = np.zeros((days_sliding_window, num_features))

#             # Loop over each day in the sequence
#             for day_idx in range(days_sliding_window):
#                 # Extract once-only features (assumed to be the same for all items)
#                 once_features = df.iloc[start_ind + day_idx * NUM_ITEMS][once_only_features].to_numpy() #müsste damit 7 Einträge haben

#                 # Start filling the day_sequences with once-only features
#                 day_sequences[day_idx, :len(once_only_features)] = once_features

#                 # Loop over each item for repeated features
#                 for item_idx in range(NUM_ITEMS):
#                     item_start_ind = start_ind + item_idx + day_idx * NUM_ITEMS
#                     item_features = df.iloc[item_start_ind:item_start_ind + 1][repeated_features].to_numpy().flatten()

#                     # Calculate the start index in the day_sequences array for this item's features
#                     features_start_idx = len(once_only_features) + item_idx * len(repeated_features)

#                     # Place the item's features in the day_sequences array
#                     day_sequences[day_idx, features_start_idx:features_start_idx + len(repeated_features)] = item_features

#             # Add the sequence for this window to batch_sequences
#             batch_sequences.append(day_sequences)

#             # Extract and add the target for this window
#             batch_target = df.iloc[end_ind:end_ind + NUM_ITEMS][target].to_numpy().flatten()
#             batch_targets.append(batch_target)

#             # Yield the batch
#             yield np.array(batch_sequences), np.array(batch_targets)



def lstm_data_generator(df, num_features, target, days_sliding_window, batch_size, once_only_features, repeated_features):
    length_days = len(df) // NUM_ITEMS  # 1941 days
    while True:
        for i in range(0, length_days - days_sliding_window):
            start_ind = i * NUM_ITEMS
            end_ind = start_ind + NUM_ITEMS * (days_sliding_window)  # predict the next day after the sequence

            # Extract once-only features for all days in the sequence at once
            once_features = df.iloc[start_ind:end_ind:NUM_ITEMS][once_only_features].to_numpy()
            # once_features = np.tile(once_features, (NUM_ITEMS, 1, 1)).transpose(1, 0, 2)

            # Extract repeated features for all items and days at once
            repeated_features_stack = df.iloc[start_ind:end_ind][repeated_features].to_numpy() # 210,000 items, 10 features

            # Reshape to a 3D array: 7 days, 30,000 items per day, 10 features
            reshaped_3d = repeated_features_stack.reshape(days_sliding_window, NUM_ITEMS, len(repeated_features))

            # Reshape to a 2D array: 7 days, 30,000 items * 10 features each
            final_array = reshaped_3d.reshape(days_sliding_window, -1)

            # Combine once-only and repeated features
            batch_sequences = np.concatenate((once_features, final_array), axis=1)

            # Reshape batch_sequences to match LSTM input shape
            batch_sequences = batch_sequences.reshape(1, days_sliding_window, -1)

            # Extract targets
            batch_targets = df.iloc[end_ind:end_ind + NUM_ITEMS][target].to_numpy()

            # Yield the batch
            yield batch_sequences, batch_targets

In [10]:
# Initialize the generator
repeated_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'sales_amount', 'sell_price', 'is_available'] # List to hold all feature columns that are used for each item
once_only_features = ['d', 'wday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'mday', 'week', 'month', 'year', 'snap_CA', 'snap_TX', 'snap_WI'] # List to hold feature columns that are not repeated for each item
num_features = len(once_only_features) + len(repeated_features) * NUM_ITEMS # Calculate the number of features

train_generator = lstm_data_generator(df_train, num_features, target_col, days_sliding_window=DAYS_PER_SEQUENCE, batch_size=1, once_only_features=once_only_features, repeated_features=repeated_features)
val_generator = lstm_data_generator(df_val, num_features, target_col, days_sliding_window=DAYS_PER_SEQUENCE, batch_size=1, once_only_features=once_only_features, repeated_features=repeated_features)

In [11]:
length_days = len(df_val) // NUM_ITEMS  # 1941 days
print(length_days - 7)

28


In [12]:
# next iter(generator_train) to see what it looks like
# generator_train_out = next(generator_train)

In [13]:
# For testing purposes: check how large on batch is
# next train_generator
# x, y = next(train_generator)
# # size of memory in mb of x and y
# print(x.nbytes / 1e6)
# print(y.nbytes / 1e6)

# print(x.shape)
# print(y.shape)

In [14]:
# # list all columns in df_train
# df_train.columns

# # call head of df_train displaying all columns without truncation
# pd.set_option('display.max_columns', None)
# df_train.head()

In [15]:
# Model parameters
epochs = 2
batch_size = 1

In [16]:
# Custom RMSE loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [17]:
# This is a sequence-to-sequence model: errors can propagate through the sequence
# model = Sequential()

# model.add(LSTM(units=30,
#                activation='tanh', #relu
#                return_sequences=False,
#                stateful=True))

# model.add(RepeatVector(28))

# model.add(LSTM(units=30, 
#                activation='tanh', 
#                return_sequences=True, 
#                stateful=True))

# model.add(Dense(units=1))

# model.compile(optimizer='adam', loss='mean_squared_error')

In [18]:
# Neu: Architecture to setup when predicting single day steps ahead and not using the repeat vector
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(units=50, 
               activation='tanh', 
               return_sequences=True, 
               stateful=True,
               input_shape=(DAYS_PER_SEQUENCE, num_features),
               batch_input_shape=(batch_size, DAYS_PER_SEQUENCE, num_features)))
model.add(Dropout(0.2))

# Additional LSTM Layer
model.add(LSTM(units=30, 
               activation='tanh', 
               return_sequences=False, 
               stateful=True))
model.add(Dropout(0.2))

# Dense layer
model.add(Dense(units=80, 
                activation='relu'))

# Final Dense layer for output
model.add(Dense(units=NUM_ITEMS))

# Reshape the output to be (number of items)
model.add(Reshape((NUM_ITEMS,))) # Eigentlich müsste das vorherige Layer bereits die richtige Form haben

model.compile(optimizer='adam', 
              loss='mse', 
              metrics=[RootMeanSquaredError()])

2024-02-02 20:36:11.306489: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14606 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:1e.0, compute capability: 7.0


In [19]:
# For tracking purposes: check the models parameters
model.summary()

# Print input shape of the layers
# for layer in model.layers:
#     print(layer.input_shape)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (1, 7, 50)                48796800  
                                                                 
 dropout (Dropout)           (1, 7, 50)                0         
                                                                 
 lstm_1 (LSTM)               (1, 30)                   9720      
                                                                 
 dropout_1 (Dropout)         (1, 30)                   0         
                                                                 
 dense (Dense)               (1, 80)                   2480      
                                                                 
 dense_1 (Dense)             (1, 30490)                2469690   
                                                                 
 reshape (Reshape)           (1, 30490)                0

In [20]:
class ResetStatesCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.model.reset_states()

In [21]:
# Training the model
history = model.fit(x=train_generator,
          steps_per_epoch=TRAIN_DUR,  # total number of sequences in the training set
          validation_data=val_generator,
          validation_steps=VAL_DUR,  # total number of sequences in the validation set
          epochs=epochs,
          callbacks=[ResetStatesCallback()])

Epoch 1/2


2024-02-02 20:36:16.104628: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2024-02-02 20:36:16.384988: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5af8017b10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-02 20:36:16.385032: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-02-02 20:36:16.392055: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-02 20:36:16.554537: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1878/1878 [==============================] - 146s 75ms/step - loss: 2.6232e-05 - root_mean_squared_error: 0.0051 - val_loss: 2.2208e-05 - val_root_mean_squared_error: 0.0047
Epoch 2/2
1878/1878 [==============================] - 140s 74ms/step - loss: 2.5772e-05 - root_mean_squared_error: 0.0051 - val_loss: 2.2220e-05 - val_root_mean_squared_error: 0.0047


In [ ]:
# Train and validation df not needed anymore
del df_train
del df_val

In [22]:
# Save the model to a specified directory
if CODE_ENV=='local':
    ###local###
    model.save(src_dir + 'models/' + MODEL_NAME + '.h5')
    
if CODE_ENV=='kaggle':
    ###On Kaggle###
    model.save('/kaggle/working/' + MODEL_NAME + '.h5')

if CODE_ENV=='aws':
    ###aws###
    model.save(src_dir + 'models/' + MODEL_NAME + '.h5')

/home/ubuntu/projects/Kaggle_M5PointPrediction/env/final_bsc/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Start from here if you want to load the model
from keras.models import load_model

# Load the model from a specified directory
if CODE_ENV=='local':
    ###local###
    model = load_model(src_dir + 'models/' + MODEL_NAME + '.h5', custom_objects={'rmse': rmse})

if CODE_ENV=='kaggle':
    ###On Kaggle###
    model = load_model('/kaggle/input/v1-model/' + MODEL_NAME + '.h5', custom_objects={'rmse': rmse})

if CODE_ENV=='aws':
    ###aws###
    model.save(src_dir + 'models/' + MODEL_NAME + '.h5', custom_objects={'rmse': rmse})

In [ ]:
import matplotlib.pyplot as plt
try:
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
except:
    print('No history to plot')

In [ ]:
# def prepare_forecast_input(df, DAYS_PER_SEQUENCE, num_items):
#     #df_test starts at 1942-7 which we need take into account
#     # Prepare input data for forecasting
#     forecast_input = []
#     for target_day in range(28):
#         start_idx = target_day * num_items
#         end_idx = start_idx + DAYS_PER_SEQUENCE * num_items
#         sequence = df.iloc[start_idx:end_idx].drop('sales_amount', axis=1).to_numpy()
#         forecast_input.append(sequence)
#     return np.array(forecast_input)


# Custom function for input to prepare forecasts input for model
# def prepare_forecast_input(df, target, model, DAYS_PER_SEQUENCE, num_items):
#     forecast_output = []
#     for target_day in range(28):
#         start_idx = target_day * num_items
#         end_idx = start_idx + DAYS_PER_SEQUENCE * num_items
#         sequence = df.iloc[start_idx:end_idx, : ].drop(target, axis=1).to_numpy()
#         # forecast_output.append(model.predict(sequence))
#         forecast_output.append(model.predict(sequence.reshape(1, sequence.shape[0], sequence.shape[1])))
#     return np.array(forecast_output)#.reshape(-1, 1)
# forecast_output = prepare_forecast_input(df_test, target_col, model, DAYS_PER_SEQUENCE, NUM_ITEMS)
#forecasts_original = scaler.inverse_transform(forecast_output)



In [ ]:
# Assuming df_all_data contains all data up to day 1941
# forecast_input = prepare_forecast_input(df_test, DAYS_PER_SEQUENCE, NUM_ITEMS)

# Generate forecasts
# forecasts = model.predict(forecast_input)
# forecasts_original = scaler.inverse_transform(forecasts)

# forecasts_original now contains the predicted sales amounts for days 1942 to 1969


In [ ]:
# Prepare input for forecasts
# I cannot use the custom lstm_data_generator
# Prepare 7 day slices each shifted by one day
def prepare_forecast_input(df, DAYS_PER_SEQUENCE, target_col):
    forecast_input = []
    for i in range(0, len(df)//NUM_ITEMS): #i=0; 1, 2, 3, ..., 35?
        if i + DAYS_PER_SEQUENCE < (len(df)-1)//NUM_ITEMS: #7, 8, 9, 10, ...
            start_idx = i*NUM_ITEMS
            end_idx   = start_idx + NUM_ITEMS * DAYS_PER_SEQUENCE
            sequence  = df.iloc[start_idx : end_idx, :].drop(target_col, axis=1).to_numpy()
            forecast_input.append(sequence)
    return np.array(forecast_input)

predict_array = prepare_forecast_input(df=df_test, DAYS_PER_SEQUENCE=DAYS_PER_SEQUENCE, target_col=target_col)

In [ ]:
predict_array.shape

In [ ]:
forecast_normalized = model.predict(predict_array)

In [ ]:
forecasts_original = scaler.inverse_transform(forecast_normalized)

In [ ]:
forecasts_original.shape

In [ ]:
# Now, let's define a function to calculate WRMSSE by calculating the RMSSE for each series and then multiplying by the weights and summing them up. 
def calculate_weights(sales_data, last_n_days=28):
    # sales_data: DataFrame with columns ['item_id', 'day', 'sales']
    # Sum sales for each item over the last_n_days
    item_sales = sales_data[sales_data['day'] > sales_data['day'].max() - last_n_days].groupby('item_id')['sales'].sum()
    # Total sales for all items
    total_sales = item_sales.sum()
    # Calculate weights
    weights = item_sales / total_sales
    return weights

def rmsse(y_true, y_pred, h, y_train):
    numerator = np.sum((y_true - y_pred) ** 2) / h
    denominator = np.sum(np.diff(y_train) ** 2) / (len(y_train) - 1) # np.diff to calc the diff for consecutive elements
    return np.sqrt(numerator / denominator)

def wrmsse(y_trues, y_preds, weights, h, y_trains):
    rmsse_values = [rmsse(y_true, y_pred, h, y_train) for y_true, y_pred, y_train in zip(y_trues, y_preds, y_trains)]
    return np.sum(np.array(weights) * np.array(rmsse_values))

In [ ]:
# Evaluate the model on the test set
def evaluate_model_wrmsse(model, df_test, df_train, df_val, batch_size, DAYS_PER_SEQUENCE, n):
    test_gen = lstm_data_generator(df_test, target_col, DAYS_PER_SEQUENCE, batch_size)
    steps = max(1, len(df_test) // (batch_size * n))  # Ensure at least 1 step
    y_pred_normalized = model.predict(test_gen, steps=steps)
    y_pred_original = scaler.inverse_transform(y_pred_normalized)
    y_true_normalized = df_test[target_col].values
    y_true_original = scaler.inverse_transform(y_true_normalized)
    
    #First concatenate all elements used for training (df_train and df_val)
    y_train_all_normalized = pd.concat([df_train[target_col], df_val[target_col]], axis=0).values
    y_train_all_original = scaler.inverse_transform(y_train_all_normalized)
    
    # Reshape the predictions and actuals to separate each item's time series
    y_pred_series = [y_pred_original[i::NUM_ITEMS] for i in range(NUM_ITEMS)]
    y_true_series = [y_true_original[i::NUM_ITEMS] for i in range(NUM_ITEMS)]

    # Similarly reshape the training data for RMSSE calculation
    y_train_all_series = [y_train_all_original[i::NUM_ITEMS] for i in range(NUM_ITEMS)]

    # Check - can be deleted later on
    print('len y_pred_series: ' + len(y_pred_series))
    print('len y_true_series: ' + len(y_true_series))
    print('len y_train_all_series: ' + len(y_train_all_series))
    
    # Calculate WRMSSE
    weights = calculate_weights(sales_data)
    wrmsse_score = wrmsse(y_trues=y_true_series, y_preds=y_pred_series, weights=weights, h=28, y_trains=y_train_all_series)

    print("Test WRMSSE: ", wrmsse_score)
    
    
    
    
    
    
    
    
    
    # Calculate wrmsse score
    wrmsse_score = wrmsse(
        y_trues=y_true_original,
        y_preds=y_pred_original,
        weights=calculate_weights(sales_data),
        h=28, # forecast horizon
        y_train=y_train_all_original
    )
    print("Test WRMSSE: ", wrmsse_score)

In [ ]:
# Call the evaluate function
# evaluate_model_wrmsse(model, df_test, df_train, df_val, batch_size, DAYS_PER_SEQUENCE, VAL_END)

- TPU nutzen und direkt aufrufen
- mutiprocessing
- use tensorflow dataset
- gpu nutzen (CUDA aufrufen)
- ConvLSTM1D layer: https://keras.io/api/layers/recurrent_layers/conv_lstm1d/
- https://www.kaggle.com/code/li325040229/eda-and-an-encoder-decoder-lstm-with-9-features/notebook#Build-a-LSTM-Model-
- Wie zum laufen bekommen?:
-   <b>Encoder-Decoder Model</b> --> https://www.kaggle.com/competitions/m5-forecasting-accuracy/discussion/144243
-   https://www.kaggle.com/competitions/m5-forecasting-accuracy/discussion/144243 --> 30490 als batch input nutzen, dann aber Problem, dass scheinbar nur Abhängigkeiten von einem auf den anderen Tag getrackt werden und keine Muster zwischen Zeitsequenzen gefunden werden können
- Herangehensweise:
    - Develop one model per site.
    -  Develop one model per group of sites.
    -  Develop one model for all sites.


<br>

- Progress bars mit tqdm anzeigen
- Test, Validierung und Trainingzeitraum sollten sich nicht überlappen, ist aber ggf. der Fall?
- Ggf. zu float16 konvertieren checken, ob finaler df mit time slices dann deutlich kleiner und performance testen
- column 'd' in training df löschen?
- paralletl computing einstellen
- use_multiprocessing in keras auf true setzen (model.fit agument)
- Cross validation?
- Ensemble learning?
- brauche ich one-hot encoding für categorical features?
- Things to consider:
- dropout
- seed
- learning rate
- loss function
- optimizer
- metrics
- batch size
- epochs
- Add CNN layer
- model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(DAYS_PER_SEQUENCE, num_features)))
- model.add(MaxPooling1D(pool_size=2))
- model.add(Flatten())
- model.add(LSTM(50, activation='relu'))
- model.add(Dense(1)) / or more layers as needed
- model.compile()